In [ ]:
%matplotlib inline

import os

import mne
import numpy as np
import scipy as sp

## Data decomposition and artefact removal with independent component analysis (ICA) - the `ICA` class

ICA is a common approach for breaking a set of signals down into the underlying components, with a mixing matrix explaining how the components are combined to form the observed signals.

In practice, this is often used to isolate artefact sources from electrophysiological recordings (cardiac activity, eye movements, stimulation artefacts, etc...) and then reconstruct the signals with these unwanted sources removed.

MNE has a comprehensive toolkit for performing ICA, in particular the [`mne.preprocessing.ICA`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html) class.

### Part 1 - Data decomposition with ICA

To explore ICA in MNE, we will generate a set of source signals which we then mix together to form the sensor signals.

The source signals are:
- A 10 Hz sine wave
- An 8 Hz sawtooth wave
- Some randomly generated noise

We create a matrix of random numbers that acts as our mixing matrix, determining how the sources project into the sensor signals.

The mixing matrix has shape `(sensors, sources)`. The (3 x 3) matrix we use here means that we will project our 3 sources into 3 sensor signals.

In [ ]:
# Simulation settings
duration = 10  # seconds
sfreq = 200  # sampling rate (Hz)
np.random.seed(44)  # for reproducibility

# Timepoints of the simulated data
times = np.linspace(start=0, stop=duration, num=sfreq * duration, endpoint=False)

# Generate source signals
sources = np.array(
    [
        np.sin(2 * np.pi * times * 10),  # 10 Hz sine wave
        sp.signal.sawtooth(2 * np.pi * times * 8),  # 8 Hz sawtooth wave
        np.random.normal(0, 1, times.shape),  # Noise with normal distribution
    ]
)
source_names = ["sine", "sawtooth", "noise"]

# Generate mixing matrix of sources to sensors
mixing_matrix = np.random.rand(3, 3)

# Combine sources into sensor signals
sensors = mixing_matrix @ sources  # @ is matrix multiplication in Python
sensor_names = ["chan_1", "chan_2", "chan_3"]

**Exercises - Data decomposition with ICA**

**Exercise:** Create an [`Info`](https://mne.tools/stable/generated/mne.Info.html) object for the source signals, specifying `source_names` as the channel names, the channel types as EEG, and using the sampling frequency we specified above.

Afterwards, use the `sources` array and `Info` object to create a [`RawArray`](https://mne.tools/stable/generated/mne.io.RawArray.html) object for the source signals, called `raw_sources`.

*Hint:* use the [`create_info()`](https://mne.tools/stable/generated/mne.create_info.html) function to create the `Info` object.

In [ ]:
## CODE GOES HERE

**Exercise:** Create an `Info` object for the sensor signals, specifying `sensor_names` as the channel names, the channel types as EEG, and using the sampling frequency we specified above.

Afterwards, use the `sensors` array and `Info` object to create a `RawArray` object for the sensor signals, called `raw_sensors`.

In [ ]:
## CODE GOES HERE

If we plot the sources, we can clearly see the individual sine wave, sawtooth, and noise channels.

In [ ]:
# The object containing the source data should be called `raw_sources`
raw_sources.plot(scalings="auto", title="Source signals");

In contrast, thanks to the mixing matrix, no one sensor signal resembles any one of the source signals.

Instead, the signals are a mix of all 3 sources.

In [ ]:
# The object containing the sensor data should be called `raw_sensors`
raw_sensors.plot(scalings="auto", title="Sensor signals");

Now consider that the sine and sawtooth signals are our signals of interest, and the random noise is some activity we are not interested in (i.e. noise!) and want to remove.

This is a perfect use-case for ICA!

#### Performing ICA

To perform ICA in MNE, we start by instantiating an [`ICA`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html) object.

Here, we set the `random_state` parameter for reproducibility, as ICA fitting is not deterministic (i.e. there can be sign flips, components returned in different orders).

In [ ]:
# Instantiate ICA object
ica = mne.preprocessing.ICA(random_state=0)
ica

As you can see, without specifying anything else, MNE defaults to using the FastICA algorithm (`Method` tab), with a set of default fitting parameters (`Fit parameters` tab).

The algorithm to use is specified with the `method` parameter, and the fitting parameters with the `fit_params` parameter.

You may also notice that we did not supply any data when instantiating the `ICA` object, and that the `Fit` tab is set to `no` (i.e. not fitted to data).

Data is provided to the `ICA` object when we call the [`fit()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.fit) method.

When calling `fit()`:
1. The data is [whitened](https://mne.tools/stable/documentation/glossary.html#term-whitening).
2. The ICA algorithm is run to generate an unmixing matrix, with which we can separate the sources in the data.

In [ ]:
# Fit ICA to the data
ica.fit(inst=raw_sensors)

We can now see that the `Fit` tab has changed to show that the ICA object has been fit to the data, and that we have 3 different ICA components available.

We can inspect the extracted ICA sources using the [`plot_sources()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.plot_sources) method.

Comparing these extracted sources to the original sources, we can clearly see that ICA has successfully separated the sine, sawtooth, and noise signals.

In [ ]:
# Plot extracted sources
ica.plot_sources(inst=raw_sensors, title="ICA sources");

We are able to extract these sources from the sensor signals thanks to the unmixing matrix, which is applied to the data provided to the `plot_sources()` method.

The unmixing matrix can be accessed under the `unmixing_matrix_` attribute (and inverse mixing matrix located under the `mixing_matrix_` attribute).

In [ ]:
# Visualise unmixing matrix
ica.unmixing_matrix_

#### Applying PCA for ICA

Before we explore how to remove the noise source, we will explore the object instantiation and fitting options in more detail.

An important implementation note for ICA in MNE is that when `fit()` is called, principal component analysis (PCA) is performed prior to running the ICA algorithm.

PCA is a well-established algorithm for dimensionality reduction, whereby correlated signals are grouped together into components that are ordered according to how much they explain the variance in the data.

You can then take only the first `n` principal components that contain a desired amount of variance (information) in the data, representing the data in a lower-dimensional space.

If you need a refresher on PCA, check out this short introductory video: https://www.youtube.com/watch?v=FD4DeN81ODY

The benefits of performing PCA prior to ICA include:
- Reduced computational time for the ICA algorithm.
- Easier interpretability of the resulting extracted ICA sources.

When instantiating the `ICA` object, MNE gives you the option to specify the degree of dimensionality reduction prior to performing ICA, using the `n_components` parameter.

##### Using PCA with a fixed number of components

**Exercises - Applying PCA for ICA**

**Exercise:** Instantiate an `ICA` object, specifying that 3 PCA components should be used for ICA.

Fit this to the sensor signals (as above), and plot the extracted sources.

How do the extracted sources compare to those where no PCA components were specified?

*Hint:* Set `random_state=0` for reproducibility.

In [ ]:
## CODE GOES HERE

You should see that the extracted sources are identical.

This is because the default behaviour of `n_components=None` means that those PCA components which explain 99.9999% of the variance in the data will be used, which will almost always correspond to the number of sensors in the data (an exception being when you are working with rank-deficient data).

Therefore, not specifying `n_components` is equivalent to specifying `n_components=3` in this case.

**Exercise:** Perform the same procedure again, but this time specify that 2 PCA components should be used for ICA.

What do you see when you plot the extracted sources?

In [ ]:
## CODE GOES HERE

##### Using PCA with a proportional number of components

In addition to specifying a particular number of PCA components to use, `n_components` also accepts floats in the range `(0, 1)`.

Providing a float value means that the number of PCA components used will be the minimum number required to explain this proportion of variance.

E.g. `n_components=0.9` means that the number of PCA components used will be the minimum number required to explain 90% of the variance in the data.

**Exercise:** Perform the same procedure again, but this time specify that 95% of the variance should be explained by the PCA components used for ICA.

In [ ]:
## CODE GOES HERE

In this case, only 2 PCA components were passed to the ICA algorithm, meaning the first 2 PCA components explain at least 95% of the variance in the data.

To see how much variance each PCA component explains, we can use the `explained_variance_ratio_` attribute.

Dividing by the sum of the variances normalises these values to the range `[0, 1]`.

Here, the first 2 PCA components explain 97% of the variance.

In [ ]:
# Compute variance explained by PCA components
explained_variance = ica.pca_explained_variance_ / np.sum(ica.pca_explained_variance_)

print(f"Variance explained by PCA components: {explained_variance}")
print(f"Variance explained by first 2 PCA components: {np.sum(explained_variance[:2]) * 100:.2f}%")

#### Excluding ICA components

Now we will look at how to exclude a given ICA component from the data.

We can remove ICA components to "clean" the data using the [`apply()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.apply) method.

Below, we specify the first component (i.e. the random noise source) to be removed from the data, by setting `exclude=[0]`.

Note that we use a copy of the sensor signals, as the `apply()` method operates in-place.

In [ ]:
# Re-instantiate the ICA object, using all PCA components
ica = mne.preprocessing.ICA(random_state=0)

# Fit the ICA to the sensor signals
ica.fit(inst=raw_sensors)

# Remove the first ICA component (the random noise) from the data
raw_cleaned = ica.apply(inst=raw_sensors.copy(), exclude=[0])

# Plot the cleaned data
raw_cleaned.plot(scalings="auto", title="Cleaned sensor signals (removed noise)");

As you can see, the remaining activity in the 3 sensor signals is a combination of our sine and sawtooth waves.

**Exercises - Choosing which sources to retain for data reconstruction**

**Exercise:** Use ICA to remove the sine wave source from the sensor signals with the `exclude` parameter.

In [ ]:
## CODE GOES HERE

**Exercise:** Use ICA to remove the sawtooth wave source from the sensor signals with the `exclude` parameter.

In [ ]:
## CODE GOES HERE

**Exercise:** Use ICA to remove both the noise and sawtooth wave sources from the sensor signals with the `exclude` parameter.

In [ ]:
## CODE GOES HERE

For convenience, the `apply()` function also has an `include` parameter which operates in the opposite way.

**Exercise:** Use ICA to keep only the random noise source in the sensor signals with the `include` parameter.

In [ ]:
## CODE GOES HERE

**Exercise:** Use ICA to keep both the sine and sawtooth wave sources in the sensor signals with the `include` parameter.

In [ ]:
## CODE GOES HERE

As you can see, it is very easy in MNE to apply ICA to data and remove particular sources of unwanted activity.

Now, you will see how this applies for a more typical use-case with the MNE sample dataset.

### Part 2 - Artefact rejection with ICA

Using the MNE sample dataset, we will see how ICA can be used to remove cardiac and ocular artefacts from MEG & EEG data.

Like for the previous notebook, we highlight some channels where this activity is particularly strong, with the MEG channels showing strong cardiac activity, and the EEG channels showing strong ocular activity (see e.g. https://labeling.ucsd.edu/tutorial/labels).

In [ ]:
# Load the sample data
raw = mne.io.read_raw_fif(
    os.path.join(mne.datasets.sample.data_path(), "MEG", "sample", "sample_audvis_raw.fif")
)
raw.crop(tmax=60)
raw.load_data()
raw.del_proj()  # delete existing PCA projections

# Highpass filter at 1 Hz for better ICA performance
raw.filter(l_freq=1, h_freq=None)

# Pick some channels with strong artefacts and plot them
artefact_picks = [152, 155, 158, 170, 315, 316, 317, 318]
raw.plot(order=artefact_picks, scalings="auto");

#### Manual exclusion of artefacts

Now we perform ICA on the data, specifying that the first 10 PCA component should be passed to the ICA algorithm.

Plotting the extracted ICA sources, we see that the first source reflects ocular activity, and the second source reflects cardiac activity.

In [ ]:
# Fit ICA to the data
ica = mne.preprocessing.ICA(n_components=10, random_state=0)
ica.fit(inst=raw)
ica.plot_sources(inst=raw, title="ICA sources (first 10 PCA components)");

Using the [`plot_overlay()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.plot_overlay) method, we can see show excluding the ocular artefact source (the first ICA component) would affect the EEG data.

In [ ]:
# Visualise effects of ocular artefact on EEG data
ica.plot_overlay(inst=raw, exclude=[0], picks="eeg");

We can also see how excluding the cardiac artefact source (the second ICA component) would affect the MEG data.

In [ ]:
# Visualise effects of cardiac artefact on MEG data
ica.plot_overlay(inst=raw, exclude=[1], picks="mag");

After this, it is simply a case of excluding the first 2 ICA components to clean the data of cardiac and ocular artefacts.

In [ ]:
# Remove artefact sources from the data
raw_cleaned = ica.apply(inst=raw.copy(), exclude=[0, 1])
raw_cleaned.plot(order=artefact_picks, scalings="auto");

#### Automatic exclusion of artefacts

Although manually selecting the components is quite simple, when dealing with a large number of recordings, this can be a time-consuming process.

Thankfully, the `ICA` class has the [`find_bads_eog()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.find_bads_eog) and [`find_bads_ecg()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.find_bads_ecg) methods, which automatically detect the ICA components corresponding to ocular and cardiac artefacts, respectively.

**N.B.** There are also the [`find_bads_muscle()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.find_bads_muscle) and [`find_bads_ref()`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html#mne.preprocessing.ICA.find_bads_ref) methods for automatically detecting muscle and MEG reference artefacts, respectively.

Using `find_bads_eog()`, we can see that the first ICA component is detected as an ocular artefact.

In [ ]:
# Automatically identify ocular artefact sources
eog_bads, _ = ica.find_bads_eog(inst=raw, threshold=1)
print(f"Ocular artefact ICA component(s): {eog_bads}")

Similarly, using `find_bads_ecg()`, we can see that the second ICA component is detected as a cardiac artefact.

In [ ]:
# Automatically identify cardiac artefact sources
ecg_bads, _ = ica.find_bads_ecg(inst=raw, threshold=0.5)
print(f"Cardiac artefact ICA component(s): {ecg_bads}")

We can then pass the ICA components that were automatically identified to `apply()` to achieve the same result as a manual selection of the artefact components.

In [ ]:
# Remove artefact sources from the data
raw_cleaned = ica.apply(inst=raw.copy(), exclude=[*eog_bads, *ecg_bads])
raw_cleaned.plot(order=artefact_picks, scalings="auto");

## Conclusion

ICA is a common approach for artefact rejection with electrophysiological data. MNE's `ICA` class provides a comprehensive set of tools for:
- Isolating unwanted sources of activity.
- Visualising the effects of removing particular sources.
- Visualising the spatial topographies of the extracted sources.
- Removing (manually or automatically) artefact activity.

## Additional resources

MNE tutorial on ICA for artefact correction: https://mne.tools/stable/auto_tutorials/preprocessing/40_artifact_correction_ica.html

arXiv paper discussing the maths behind ICA: https://arxiv.org/pdf/1404.2986.pdf